<a href="https://colab.research.google.com/github/junggeyy/DeepLearning/blob/main/multilayer_perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementing a Multilayer Perceptron (XOR)



In [ ]:
# !pip install torch scikit-learn

## Loading the Dataset

In [13]:
import pandas as pd

df = pd.read_csv("xor.csv")
df

,x1,x2,class label
0,0.781306,1.062984,0
1,-1.060524,-1.095550,0
2,0.632125,0.674028,0
3,-1.424712,0.535203,1
4,1.383161,1.368510,0
...,...,...,...
745,0.792484,0.839275,0
746,0.582466,-0.749250,1
747,-1.593475,0.671721,1
748,-0.812671,-0.268542,0


In [14]:
X = df[["x1", "x2"]].values
y = df["class label"].values

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15,
                                                    random_state=1, stratify=y)

In [17]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=1, stratify=y_train)

In [18]:
print("Training size: ", X_train.shape)
print("Validation size: ", X_val.shape)
print("Test size: ", X_test.shape)

Training size:  (573, 2)
Validation size:  (64, 2)
Test size:  (113, 2)


In [20]:
# testing if we have a evenly distributed labels
import numpy as np

print("Training labels: ", np.bincount(y_train))
print("Validation labels: ", np.bincount(y_val))
print("Test labels: ", np.bincount(y_test))

Training labels:  [287 286]
Validation labels:  [32 32]
Test labels:  [57 56]


## Implementing the Model

In [29]:
import torch

class PyTorchMLP(torch.nn.Module):
  def __init__(self, num_features, num_classes):
    super().__init__()

    self.all_layers = torch.nn.Sequential(

        # 1st hidden layer
        torch.nn.Linear(num_features, 25),
        torch.nn.ReLU(),

        #2nd hidden layer
        torch.nn.Linear(25, 15),
        torch.nn.ReLU(),

        #output layer
        torch.nn.Linear(15, num_classes),
    )

  def forward(self, x):
    logits = self.all_layers(x)
    return logits

## Defining the DataLoader

In [34]:
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
  def __init__(self, X, y):
    self.features = torch.tensor(X, dtype=torch.float32)
    self.labels = torch.tensor(y, dtype=torch.int64)

  def __getitem__(self, index):
    x = self.features[index]
    y = self.labels[index]
    return x, y

  def __len__(self):
    return self.labels.shape[0]

train_ds = MyDataset(X_train, y_train)
test_ds = MyDataset(X_test, y_test)
val_ds = MyDataset(X_val, y_val)

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=32,
    shuffle=True
)
test_loader = DataLoader(
    dataset=test_ds,
    batch_size=32,
    shuffle=False
)
val_loader = DataLoader(
    dataset=val_ds,
    batch_size=32,
    shuffle=False
)

## Training Loop

In [35]:
def compute_accuracy(model, dataloader):
  model = model.eval()

  correct = 0.0
  total_example = 0.0

  for idx, (features, labels) in enumerate(dataloader):
    with torch.inference_mode():  # works like no_grad but more optimized
      logits = model(features)

    predictions = torch.argmax(logits, dim=1)

    compare = (predictions == labels)
    correct += torch.sum(compare)
    total_example += len(compare)

  return correct / total_example

In [ ]:
import torch.nn.functional as F

torch.manual_seed(1)
model = PyTorchMLP(num_features=2, num_classes=2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)
num_epochs = 10

for epoch in range(num_epochs):
  model = model.train()

  for batch_idx, (features, labels) in enumerate(train_loader):
    logits = model(features)
    loss = F.cross_entropy(logits, labels)  # performs softmax internally

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Logging
    print(f'Epoch: {epoch+1:03d}/{num_epochs:03d}'
          f' | Batch {batch_idx:03d}/{len(train_loader):03d}'
          f' | Loss: {loss:.2f}')

  train_accuracy = compute_accuracy(model, train_loader)
  val_accuracy = compute_accuracy(model, val_loader)
  print(f"Test Accuracy: {train_accuracy*100:.2f}%")
  print(f"Validation Accuracy: {val_accuracy*100:.2f}%")



## Evaluating the Model

In [41]:
train_accuracy = compute_accuracy(model, train_loader)
val_accuracy = compute_accuracy(model, val_loader)
test_accuracy = compute_accuracy(model, test_loader)
print(f"Test Accuracy: {train_accuracy*100:.2f}%")
print(f"Validation Accuracy: {val_accuracy*100:.2f}%")
print(f'Test Accuracy: {test_accuracy*100:.2f}%')

Test Accuracy: 98.25%
Validation Accuracy: 100.00%
Test Accuracy: 99.12%
